In [1]:
import pandas as pd
import ezdxf
import numpy as np

In [2]:
path_trm = "../calc/trm_res_plots.xlsx"
path_quad = "../calc/quad_res_plots.xlsx"
path_tho = "../calc/tho_res_plots.xlsx"
sheet_name_trm = ['lpa_0477', 'lpa_0747', 'lpa_0779', 'lpa_0833']
sheet_name_quad = ['lpa_0400', 'lpa_0500']
sheet_name_tho = ['lpa_0633', 'lpa_0568', 'lpa_0366', 'lpa_0300', 'lpa_0101']
sheet_name_tho_def = ['lpa_0633_w', 'lpa_0568_w', 'lpa_0366_w', 'lpa_0300_w', 'lpa_0101_w']

DF_nodes = pd.read_excel(path_quad, sheet_name='knoten')[['X [m]', 'Y [m]', 'Z [m]']]
DF_w = pd.read_excel(path_tho, sheet_name='lpa_0101_w')

In [3]:
def read_data(path, sheet_name):
    DF = pd.read_excel(path, sheet_name=sheet_name)[['Knoten', 'My  [kNm]', 'Tx  [kNm]','Vz  [kN]']].dropna()
    #transformation
    DF['Knoten'] = DF['Knoten'].astype(int)*-1 -1
    #Merge with nodes coordinates
    DF_res = DF.merge(DF_nodes, left_on='Knoten', right_index=True)
    #rename
    columns_rename = ['knoten', 'My', 'Tx','Vz', 'x', 'y', 'z']
    DF_res.columns = columns_rename
    #reset index
    DF_res=DF_res.reset_index()

    return DF_res

def read_data_def(path, sheet_name):
    DF = pd.read_excel(path, sheet_name=sheet_name)[['Knoten', 'ez  [mm]']].dropna()
    #transformation
    DF['Knoten'] = DF['Knoten'].astype(int)*-1 -1
    #Merge with nodes coordinates
    DF_res = DF.merge(DF_nodes, left_on='Knoten', right_index=True)
    #rename
    columns_rename = ['knoten', 'w', 'x', 'y', 'z']
    DF_res.columns = columns_rename
    #reset index
    DF_res=DF_res.reset_index()

    return DF_res

def save_dxf(Data, dxf_name, moment,scale):

    # Extract rows for DXF
    x = Data['x']
    y = Data['y']
    z = Data[moment]/scale

    # Generate consistent pairs (0-1, 2-3, ...)
    index_pairs = [(i, i + 1) for i in range(0, len(x), 2)]

    # Create a new DXF document
    doc = ezdxf.new()

    # Add a new modelspace
    msp = doc.modelspace()



    # Add 3D polyline to the DXF
    points = list(zip(x, y, z))

    # Add line segments for each pair
    for start, end in index_pairs:
        points = [(x[start],y[start],z[start]), (x[end],y[end],z[end])]
        msp.add_polyline3d(points, dxfattribs={'layer': '3D Line'})

    # Add vertical lines from z=0 to z=z[i] for each point
    for i in range(len(x)):
        base_point = (x[i], y[i], 0)  # Point at z=0
        top_point = (x[i], y[i], z[i])  # Point at z=z[i]
        msp.add_line(base_point, top_point, dxfattribs={'layer': 'Vertical Lines'})
        

    # Save to file
    output_path = dxf_name+'_'+moment+".dxf"
    doc.saveas(output_path)

    print(f"DXF file saved as {output_path}")

In [6]:
for sheet in sheet_name_quad:    
    DG_sg = read_data(path = path_quad, sheet_name=sheet)
    save_dxf(DG_sg, sheet, 'My', scale=5)
    save_dxf(DG_sg, sheet, 'Tx', scale=5)
    save_dxf(DG_sg, sheet, 'Vz', scale=150)


# for sheet in sheet_name_tho_def:    
#     DG_w = read_data_def(path = path_tho, sheet_name=sheet)
#     save_dxf(DG_w, sheet, 'w', scale=50)

DXF file saved as lpa_0400_My.dxf
DXF file saved as lpa_0400_Tx.dxf
DXF file saved as lpa_0400_Vz.dxf
DXF file saved as lpa_0500_My.dxf
DXF file saved as lpa_0500_Tx.dxf
DXF file saved as lpa_0500_Vz.dxf


In [7]:
DG_sg = read_data(path_quad, sheet_name_quad[1])

DG_sg

,index,knoten,My,Tx,Vz,x,y,z
0,1,36,43.8,0.0,-43.8,9,9,0
1,2,9,0.0,0.0,-43.8,10,9,0
2,5,37,0.0,-100.0,-56.3,8,9,0
3,6,36,-56.3,-100.0,-56.3,9,9,0
4,9,38,-50.0,-100.0,-50.0,7,9,0
...,...,...,...,...,...,...,...,...
355,710,45,-100.0,100.0,-50.0,9,8,0
356,713,45,0.0,100.0,-56.3,9,8,0
357,714,36,-56.3,100.0,-56.3,9,9,0
358,717,36,43.8,0.0,-43.8,9,9,0
